In [11]:
# use magic command to auto-reload
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import igraph as ig
import json
from dotenv import load_dotenv
import os
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from simulation import run_simulation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import dotenv
import os

# load PARAMS.json
with open("PARAMS.json", "r") as f:
    PARAMS = json.load(f)

PARAMS["time_decay_rate"] =  np.log(2)/2  # halving the weight every 2 time steps. change /2 -> /k for halving every k time steps.
PARAMS["W_agent_success"] = 1
PARAMS["W_personal_weights"] = 1
PARAMS["W_post_success"] = 1
PARAMS["noise_level"] = 0.01



ENV = dotenv.dotenv_values("./.env")
HF_TOKEN = ENV["HF_TOKEN"]
API_URL = ENV["API_URL"]
MODEL = ENV["MODEL"]


print(API_URL)
print(MODEL)

# add HF_TOKEN to PARAMS
PARAMS["HF_TOKEN"] = HF_TOKEN
PARAMS["API_URL"] = API_URL
PARAMS["MODEL"] = MODEL


timesteps = 50
num_agents = 8
fill_history = 2


PARAMS["num_agents"] = num_agents
PARAMS["timesteps"] = timesteps
PARAMS["fill_history"] = fill_history


G, POSTS, WEIGHTS, READ_MATRIX, LIKES = run_simulation(PARAMS)
print(np.mean(LIKES, axis=1))

https://router.huggingface.co/v1/chat/completions
meta-llama/Llama-3.1-8B-Instruct:cerebras
T_current: 2
T_current: 3
T_current: 4
T_current: 5
T_current: 6
T_current: 7
T_current: 8
T_current: 9
T_current: 10
T_current: 11
T_current: 12
T_current: 13
T_current: 14
T_current: 15
T_current: 16
T_current: 17
T_current: 18
T_current: 19
T_current: 20
T_current: 21
T_current: 22
T_current: 23
T_current: 24
T_current: 25
T_current: 26
T_current: 27
T_current: 28
T_current: 29
T_current: 30
T_current: 31
T_current: 32
T_current: 33
T_current: 34
T_current: 35
T_current: 36
T_current: 37
T_current: 38
T_current: 39
T_current: 40
T_current: 41
T_current: 42
T_current: 43
T_current: 44
T_current: 45
T_current: 46
T_current: 47
T_current: 48
T_current: 49
T_current: 50
T_current: 51
[0.         0.01923077 0.03846154 5.73076923 6.23076923 1.71153846
 0.03846154 0.01923077]
